# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from IPython import display
import warnings

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName;", engine)
X = df.message.values
Y = df.iloc[:, 4:].values
category_label = df.iloc[:, 4:].columns.tolist()

In [3]:
df[df.genre=='direct'].sample(15)[['genre', 'message']].values

array([['direct',
        "Everybody is gone from the earthquake. My house is crushed. I don't have any possibility to buy anything. If anyone can help me find a job. .."],
       ['direct',
        "It's not obvious to say the truth the way it is,but this is an emergency and more than emergency.If that's me that you reward,It's will smaller than the one of God.Thank's "],
       ['direct', 'Things are so difficult for me and my child. Thanks'],
       ['direct',
        "hello I am a nurse, I don't work, I spend a difficult moment in my life help me I implore of it to you "],
       ['direct',
        "Hello my brother. I ask you to do all that is in your power for us in Avenue Christophe Chanel because we've got problems of water and food, etc. "],
       ['direct', "what is today's informations? "],
       ['direct',
        'finding a little food to eat, because there are alot of us hungry'],
       ['direct',
        "My brother and my Mother died from this time there are 6 people

In [4]:
def silence_print(true_label, pred_label, silence=True):
    """Print classification report without warning message
    
    Parameters:
    -----------
    true_label: array
        The true values
    pred_label: array
        The predict values
    silence: boolean default True
        If true, print the report without warning message
    """
    if silence:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            for index, column in enumerate(category_label):
                report = classification_report(
                    true_label[:, index], pred_label[:, index], labels=np.arange(1), \
                    target_names=[column], output_dict=True, digits=2
                )
                print("{0:<25}\n  Accuracy:{1: 5.3f}  Precision:{2: =6.3f}   Recall:{3:< 5.3f}".format(
                    column, accuracy_score(true_label[:, index], pred_label[:, index]), 
                    report[column]["precision"], report[column]["recall"], report[column]["f1-score"]
                ))
    else:
        for index, column in enumerate(category_label):
            report = classification_report(
                    true_label[:, index], pred_label[:, index], labels=np.arange(1), \
                    target_names=[column], output_dict=True, digits=2
                )
            print("{0:<25}\n  Accuracy:{1: 5.3f}  Precision:{2: =6.3f}   Recall:{3:< 5.3f}".format(
                    column, accuracy_score(true_label[:, index], pred_label[:, index]), 
                    report[column]["precision"], report[column]["recall"], report[column]["f1-score"]
                ))

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """Tokenize the text 
    
    Tokenize the text information by using word_tokenize, WordNetLemmatizer, 
    remove words that is in stopwords
    
    Parameters:
    -----------
    text: string
        String contains the message information
    
    Results:
    ----------
    result: list
        List contains strings parsed from the text
    """
    # nromalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("text_pipeline", Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=2))
])

### 4. Train pipeline
* Split data into train and test sets
* Train pipeline

In [7]:
# split data
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=42)

In [8]:
# train data
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=2))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# basic model
pred_y = pipeline.predict(test_X)

In [10]:
# print the report
silence_print(test_y, pred_y)

related                  
  Accuracy: 0.792  Precision: 0.601   Recall: 0.451
request                  
  Accuracy: 0.885  Precision: 0.897   Recall: 0.973
offer                    
  Accuracy: 0.995  Precision: 0.995   Recall: 1.000
aid_related              
  Accuracy: 0.752  Precision: 0.762   Recall: 0.842
medical_help             
  Accuracy: 0.922  Precision: 0.926   Recall: 0.994
medical_products         
  Accuracy: 0.951  Precision: 0.952   Recall: 0.999
search_and_rescue        
  Accuracy: 0.976  Precision: 0.976   Recall: 1.000
security                 
  Accuracy: 0.981  Precision: 0.982   Recall: 1.000
military                 
  Accuracy: 0.970  Precision: 0.973   Recall: 0.997
child_alone              
  Accuracy: 1.000  Precision: 1.000   Recall: 1.000
water                    
  Accuracy: 0.953  Precision: 0.955   Recall: 0.996
food                     
  Accuracy: 0.936  Precision: 0.943   Recall: 0.988
shelter                  
  Accuracy: 0.929  Precision: 0.935   

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
forest = RandomForestClassifier(random_state=42)

pipeline = Pipeline([
    ("text_pipeline", Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer())
    ])),
    ("clf", MultiOutputClassifier(forest))
])

In [12]:
# params
params = {
#     'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'text_pipeline__vect__max_df': (0.75, 1.0),
    'text_pipeline__vect__max_features': (5000, 100000),
#     'text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 20, 30],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [4, 6, 10],

}

In [13]:
# train model
cv = GridSearchCV(pipeline, param_grid=params, cv=3)

cv.fit(train_X, train_y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'text_pipeline__vect__max_features': (5000, 100000), 'clf__estimator__n_estimators': [10, 20, 30], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [4, 6, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# predict values
pred_y = cv.predict(test_X)

In [15]:
# print report
silence_print(test_y, pred_y)

related                  
  Accuracy: 0.754  Precision: 0.000   Recall: 0.000
request                  
  Accuracy: 0.831  Precision: 0.831   Recall: 1.000
offer                    
  Accuracy: 0.995  Precision: 0.995   Recall: 1.000
aid_related              
  Accuracy: 0.598  Precision: 0.595   Recall: 1.000
medical_help             
  Accuracy: 0.919  Precision: 0.919   Recall: 1.000
medical_products         
  Accuracy: 0.947  Precision: 0.947   Recall: 1.000
search_and_rescue        
  Accuracy: 0.976  Precision: 0.976   Recall: 1.000
security                 
  Accuracy: 0.982  Precision: 0.982   Recall: 1.000
military                 
  Accuracy: 0.969  Precision: 0.969   Recall: 1.000
child_alone              
  Accuracy: 1.000  Precision: 1.000   Recall: 1.000
water                    
  Accuracy: 0.936  Precision: 0.936   Recall: 1.000
food                     
  Accuracy: 0.888  Precision: 0.888   Recall: 1.000
shelter                  
  Accuracy: 0.910  Precision: 0.910   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
def chunck_tokenize(text):
    """Tokenize the text 
    
    Tokenize the text information by using word_tokenize, WordNetLemmatizer, 
    remove words that is in stopwords, and use the new regular expresssiong
    
    Parameters:
    -----------
    text: string
        String contains the message information
    
    Results:
    ----------
    result: list
        List contains strings parsed from the text
    """
    # nromalize text
    text = re.sub(r"#[a-zA-Z0-9]", "tagname", text)
    text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "urlholder", text)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token.lower() not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result

In [17]:
def check(x):
    """Parse the text into more information
    Get the more information to analyze, like sentence length over 1
    and check the pos tag type
    
    Parameters:
    -----------
    x: string
        Messages string
    
    Results:
    ----------
    length: boolean
        Check the sentence length is over 1
    chunck: boolean
        Check whether there is specific pos tag
    """
    length = len(sent_tokenize(x))
    chunck = False

    for sentence in sent_tokenize(x):
        pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

        try:
            word, tag = pos_tags[0]

            if "VB" in tag:
                chunck = True
        except:
            continue
        
        try:
            for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                if tag == "NNP":
                    chunck = True
                    break
        except:
            continue
            
        if chunck:
            break

    return (length, chunck)

In [18]:
class WordChunckExtract(BaseEstimator, TransformerMixin):
    """Word Pos Tag Extract
    
    Word Chunck extract the text information, which is used to 
    fit and transform method
    """
    def starting_verb(self, text):
        """Parse the text into more information
        Get the more information to analyze, like sentence length over 1
        and check the pos tag type

        Parameters:
        -----------
        text: string
            Messages string

        Results:
        ----------
        length: boolean
            Check the sentence length is over 1
        chunck: boolean
            Check whether there is specific pos tag
        """
        length = len(sent_tokenize(text))
        chunck = False

        for sentence in sent_tokenize(text):
            pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

            try:
                word, tag = pos_tags[0]

                if "VB" in tag:
                    chunck = True
            except:
                continue

            try:
                for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                    if tag == "NNP":
                        chunck = True
                        break
            except:
                continue

            if chunck:
                break

        return (length, chunck)

    def fit(self, x, y=None):
        """
        Fit the text message
        """
        return self

    def transform(self, X):
        """
        Transform the text message
        """
        result = pd.DataFrame()
        X_tagged = pd.Series(X).apply(self.starting_verb)
        result["length"] = X_tagged.apply(lambda x: x[0] > 1)
        result["chucnk"] = X_tagged.apply(lambda x: x[1])
        return result


In [19]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("features", FeatureUnion([
        ("text_pipeline", Pipeline([
            ("vect", CountVectorizer(tokenizer=tokenize)),
            ("tfidf", TfidfTransformer())
        ])), 
        ("exact_chunck", WordChunckExtract())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=4))
])

In [20]:
# params
params = {
#     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [100, 150],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [12, 18, 25],
    'features__transformer_weights': (
    {'text_pipeline': 1, 'starting_verb': 0.5},
#     {'text_pipeline': 0.5, 'starting_verb': 1},
#     {'text_pipeline': 0.8, 'starting_verb': 1},
)

}

# train model
cv = GridSearchCV(pipeline, param_grid=params, cv=3)

cv.fit(train_X, train_y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=4))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'clf__estimator__n_estimators': [100, 150], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [12, 18, 25], 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5},)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
# predict values
pred_y = cv.predict(test_X)

In [22]:
silence_print(test_y, pred_y)

related                  
  Accuracy: 0.754  Precision: 0.000   Recall: 0.000
request                  
  Accuracy: 0.831  Precision: 0.831   Recall: 1.000
offer                    
  Accuracy: 0.995  Precision: 0.995   Recall: 1.000
aid_related              
  Accuracy: 0.633  Precision: 0.618   Recall: 0.994
medical_help             
  Accuracy: 0.919  Precision: 0.919   Recall: 1.000
medical_products         
  Accuracy: 0.947  Precision: 0.947   Recall: 1.000
search_and_rescue        
  Accuracy: 0.976  Precision: 0.976   Recall: 1.000
security                 
  Accuracy: 0.982  Precision: 0.982   Recall: 1.000
military                 
  Accuracy: 0.969  Precision: 0.969   Recall: 1.000
child_alone              
  Accuracy: 1.000  Precision: 1.000   Recall: 1.000
water                    
  Accuracy: 0.936  Precision: 0.936   Recall: 1.000
food                     
  Accuracy: 0.888  Precision: 0.888   Recall: 1.000
shelter                  
  Accuracy: 0.910  Precision: 0.910   

### 9. Export your model as a pickle file

In [23]:
import pickle

In [24]:
with open("model.pickle", "wb") as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.